# Cuisine Classification ML Training Pipeline

This notebook demonstrates the complete ML training pipeline for cuisine classification using the gold layer data from our medallion architecture. It follows Databricks MLflow best practices for experiment tracking, model registration, and serving.

## Pipeline Overview
1. **Data Preparation**: Load gold layer datasets with feature vectors
2. **Model Training**: Fine-tune ResNet-50 for cuisine classification  
3. **MLflow Integration**: Track experiments and register models
4. **Model Serving**: Deploy model for real-time inference
5. **Batch Inference**: Process new images with trained model

In [ ]:
# Install required packages for ML training
%pip install datasets==2.20.0 transformers==4.49.0 tf-keras==2.17.0 accelerate==1.4.0 mlflow==2.20.2 torchvision==0.20.1 deepspeed==0.14.4
dbutils.library.restartPython()

In [ ]:
import mlflow
import torch
from datasets import Dataset
import pandas as pd
import numpy as np
from transformers import AutoFeatureExtractor, AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer
from transformers import pipeline, DefaultDataCollator, EarlyStoppingCallback
from PIL import Image
import io
from torchvision.transforms import CenterCrop, Compose, Normalize, RandomResizedCrop, Resize, ToTensor, Lambda
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient
from pyspark.sql.functions import col, pandas_udf
from typing import Iterator
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
CATALOG = "cuisine_vision_catalog"
GOLD_SCHEMA = "gold"
IMAGE_SIZE = 224
MODEL_CHECKPOINT = "microsoft/resnet-50"
EXPERIMENT_NAME = "/Users/databricks/cuisine-classification-pipeline"

print("🚀 Starting Cuisine Classification ML Pipeline")
print(f"📊 Using catalog: {CATALOG}")
print(f"🥇 Gold schema: {GOLD_SCHEMA}")
print(f"🖼️ Image size: {IMAGE_SIZE}x{IMAGE_SIZE}")
print(f"🧠 Base model: {MODEL_CHECKPOINT}")

## 1. Data Preparation from Gold Layer
Load the ML-ready dataset with proper stratified splits and feature vectors.

## 1.1 Feature Store Setup
Setup Databricks Feature Store for advanced feature management and discovery.

In [ ]:
# Setup Databricks Feature Store for advanced feature management
from databricks.feature_store import FeatureStoreClient, FeatureLookup
from pyspark.sql.functions import col

# Initialize Feature Store client
fs = FeatureStoreClient()

# Feature Store configuration
FEATURE_STORE_SCHEMA = "cuisine_vision_catalog.features"
FEATURE_TABLE_NAME = f"{FEATURE_STORE_SCHEMA}.image_features"

print("🏪 Setting up Databricks Feature Store...")
print(f"   📊 Feature table: {FEATURE_TABLE_NAME}")

def setup_feature_store():
    """One-time setup: Create Feature Store table from gold layer features"""
    try:
        # Read feature vectors from gold layer
        feature_df = spark.table(f"{CATALOG}.{GOLD_SCHEMA}.feature_vectors")
        
        print(f"📊 Creating Feature Store table: {FEATURE_TABLE_NAME}")
        
        # Create feature table (ONE-TIME OPERATION)
        fs.create_feature_table(
            name=FEATURE_TABLE_NAME,
            primary_keys=["image_id"],
            df=feature_df.limit(1),  # Just for schema inference
            description="Advanced image feature vectors for cuisine classification ML models",
            tags={
                "team": "data-science",
                "use_case": "cuisine_classification", 
                "feature_type": "image_features",
                "version": "v1",
                "pipeline": "cuisine_vision_dlt"
            }
        )
        
        print(f"✅ Feature Store table created: {FEATURE_TABLE_NAME}")
        return True
        
    except Exception as e:
        if "already exists" in str(e).lower():
            print(f"ℹ️  Feature table already exists: {FEATURE_TABLE_NAME}")
            return True
        else:
            print(f"❌ Error creating feature table: {str(e)}")
            return False

def update_feature_store():
    """Update Feature Store with latest feature data"""
    try:
        # Read latest feature vectors
        feature_df = spark.table(f"{CATALOG}.{GOLD_SCHEMA}.feature_vectors")
        
        print(f"🔄 Updating Feature Store with {feature_df.count()} feature records...")
        
        # Write features to Feature Store
        fs.write_table(
            name=FEATURE_TABLE_NAME,
            df=feature_df,
            mode="overwrite"  # Use 'merge' for incremental updates
        )
        
        print("✅ Feature Store updated successfully!")
        return True
        
    except Exception as e:
        print(f"❌ Error updating Feature Store: {str(e)}")
        return False

# Setup Feature Store (run once)
if setup_feature_store():
    update_feature_store()
    print("\n🏪 Feature Store Benefits:")
    print("   📈 Feature Discovery: Browse features in Databricks UI")
    print("   🔗 Feature Lineage: Track feature origin and transformations")
    print("   ⚡ Feature Serving: Fast feature lookups for inference") 
    print("   🔄 Feature Reuse: Share features across ML teams")
    print("   📊 Feature Monitoring: Track feature drift and quality")

In [ ]:
# Load gold layer ML dataset
print("📋 Loading gold layer ML dataset...")

ml_dataset = spark.table(f"{CATALOG}.{GOLD_SCHEMA}.ml_dataset")
# feature_vectors = spark.table(f"{CATALOG}.{GOLD_SCHEMA}.feature_vectors")  # Optional for advanced features

print(f"✅ Dataset loaded:")
print(f"   📊 Total records: {ml_dataset.count()}")
print(f"   🌍 Unique cuisines: {ml_dataset.select('cuisine_category').distinct().count()}")
print(f"   🍽️ Unique food types: {ml_dataset.select('food_type').distinct().count()}")

# Show dataset distribution
print("\n📈 Dataset split distribution:")
ml_dataset.groupBy("dataset_split", "cuisine_category").count().orderBy("cuisine_category", "dataset_split").show()

In [ ]:
# Option A: Use Feature Store for training (enhanced features)
print("🏪 Feature Store Integration Option...")

try:
    # Create training set with Feature Store lookups
    # Get base data with labels and image_id
    base_training_df = (
        spark.table(f"{CATALOG}.{GOLD_SCHEMA}.ml_dataset")
        .filter(col("dataset_split") == "train")
        .select("image_id", "cuisine_category", "processed_image_data")
    )
    
    # Create Feature Store training set
    feature_lookups = [
        FeatureLookup(
            table_name=FEATURE_TABLE_NAME,
            lookup_key="image_id"
        )
    ]
    
    training_set = fs.create_training_set(
        df=base_training_df,
        feature_lookups=feature_lookups,
        label="cuisine_category"
    )
    
    # Get enhanced training dataframe with features
    enhanced_training_df = training_set.load_df()
    
    print("✅ Feature Store training set created!")
    print(f"   📊 Features available: {len(enhanced_training_df.columns)} columns")
    print(f"   🧠 100D feature vector: feature_vector_v1")
    print(f"   🎨 Color features: color_histogram, dominant_colors, etc.")
    print(f"   🔲 Texture features: edge_density, texture_contrast, etc.")
    print(f"   📐 Shape features: symmetry_score, structural_complexity")
    
    use_feature_store = True
    
except Exception as e:
    print(f"ℹ️  Feature Store not available, using standard approach: {str(e)}")
    use_feature_store = False

if not use_feature_store:
    print("📊 Using standard gold layer data...")
    # Fallback to standard approach
    enhanced_training_df = ml_dataset.filter(col("dataset_split") == "train")

In [ ]:
# Get training data with proper format for HuggingFace
print("🔄 Preparing training data for HuggingFace...")

# Select training data with images and labels
training_data = (
    ml_dataset
    .filter(col("dataset_split") == "train")
    .select("image_id", "processed_image_data", "cuisine_category", "cuisine_category_encoded", "food_type")
    .filter(col("processed_image_data").isNotNull())
)

validation_data = (
    ml_dataset
    .filter(col("dataset_split") == "validation") 
    .select("image_id", "processed_image_data", "cuisine_category", "cuisine_category_encoded", "food_type")
    .filter(col("processed_image_data").isNotNull())
)

test_data = (
    ml_dataset
    .filter(col("dataset_split") == "test")
    .select("image_id", "processed_image_data", "cuisine_category", "cuisine_category_encoded", "food_type") 
    .filter(col("processed_image_data").isNotNull())
)

print(f"✅ Data splits prepared:")
print(f"   🏋️ Training: {training_data.count()} samples")
print(f"   ✅ Validation: {validation_data.count()} samples") 
print(f"   🧪 Test: {test_data.count()} samples")

In [ ]:
# Convert to HuggingFace Dataset format
print("🤗 Converting to HuggingFace Dataset format...")

# Convert Spark DataFrames to HuggingFace Datasets
train_dataset = Dataset.from_spark(
    training_data, 
    cache_dir="/tmp/hf_cache/train"
).rename_column("processed_image_data", "image").rename_column("cuisine_category", "label")

val_dataset = Dataset.from_spark(
    validation_data,
    cache_dir="/tmp/hf_cache/val"
).rename_column("processed_image_data", "image").rename_column("cuisine_category", "label")

test_dataset = Dataset.from_spark(
    test_data,
    cache_dir="/tmp/hf_cache/test"  
).rename_column("processed_image_data", "image").rename_column("cuisine_category", "label")

print(f"✅ HuggingFace datasets created:")
print(f"   📊 Training features: {train_dataset.features}")
print(f"   📈 Training samples: {len(train_dataset)}")
print(f"   📈 Validation samples: {len(val_dataset)}")

## 2. Model Setup and Preprocessing
Setup the ResNet-50 model for fine-tuning with proper preprocessing pipelines.

In [ ]:
# Setup model and preprocessing
print("🧠 Setting up model and preprocessing...")

# Load feature extractor for preprocessing
feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_CHECKPOINT)

# Define image transformations
transforms = Compose([
    Lambda(lambda b: Image.open(io.BytesIO(b)).convert("RGB")),  # Byte to PIL
    ToTensor(),  # Convert PIL to tensor
    Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
])

def preprocess_images(batch):
    """Apply transformations to image batch"""
    batch["image"] = [transforms(image) for image in batch["image"]]
    return batch

# Apply transformations
train_dataset.set_transform(preprocess_images)
val_dataset.set_transform(preprocess_images)
test_dataset.set_transform(preprocess_images)

print("✅ Preprocessing setup complete")

In [ ]:
# Create label mappings for model
print("🏷️ Creating label mappings...")

# Get unique labels and create mappings
all_labels = set(train_dataset['label']) | set(val_dataset['label'])
label2id = {label: i for i, label in enumerate(sorted(all_labels))}
id2label = {i: label for label, i in label2id.items()}

print(f"✅ Label mappings created:")
print(f"   🏷️ Total classes: {len(label2id)}")
print(f"   🌍 Cuisines: {list(label2id.keys())}")

# Load model with proper configuration
model = AutoModelForImageClassification.from_pretrained(
    MODEL_CHECKPOINT,
    label2id=label2id,
    id2label=id2label,
    num_labels=len(label2id),
    ignore_mismatched_sizes=True
)

print("✅ Model loaded successfully")

## 3. MLflow Experiment Setup
Configure MLflow for experiment tracking and model management.

In [ ]:
# Setup MLflow experiment
print("🧪 Setting up MLflow experiment...")

mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.set_registry_uri("databricks-uc")

print(f"✅ MLflow experiment setup:")
print(f"   🧪 Experiment: {EXPERIMENT_NAME}")
print(f"   📊 Registry: databricks-uc")

## 4. Model Training with MLflow Tracking
Train the model with comprehensive experiment tracking.

In [ ]:
# Training configuration
model_name = MODEL_CHECKPOINT.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"/tmp/huggingface/cuisine/{model_name}-finetuned",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/tmp/logs',
    logging_steps=100,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    remove_unused_columns=False,
    dataloader_num_workers=4,
    fp16=True  # Use mixed precision for faster training
)

print("⚙️ Training configuration set")

In [ ]:
# Custom model wrapper for MLflow
class CuisineClassifierWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, pipeline):
        self.pipeline = pipeline
        self.pipeline.model.eval()

    def predict(self, context, images):
        """Predict cuisine category from image binary data"""
        from PIL import Image
        import pandas as pd
        
        with torch.set_grad_enabled(False):
            # Convert binary data to PIL images
            if 'processed_image_data' in images.columns:
                pil_images = images['processed_image_data'].apply(
                    lambda b: Image.open(io.BytesIO(b)) if b is not None else None
                ).tolist()
            else:
                pil_images = images['image'].apply(
                    lambda b: Image.open(io.BytesIO(b)) if b is not None else None
                ).tolist()
            
            # Filter out None images
            valid_images = [img for img in pil_images if img is not None]
            
            if not valid_images:
                return pd.DataFrame([{'label': 'unknown', 'score': 0.0}])
            
            # Get predictions
            predictions = self.pipeline(valid_images)
            
            # Return top prediction for each image
            results = []
            for pred in predictions:
                if isinstance(pred, list):
                    top_pred = max(pred, key=lambda x: x['score'])
                else:
                    top_pred = pred
                results.append(top_pred)
            
            return pd.DataFrame(results)

print("✅ Model wrapper defined")

In [ ]:
# Start MLflow run and train model
print("🚀 Starting model training with MLflow tracking...")

with mlflow.start_run(run_name="cuisine-classification-resnet50") as run:
    # Log dataset information
    mlflow.log_input(mlflow.data.from_huggingface(train_dataset, "training"))
    
    # Log parameters
    mlflow.log_params({
        "model_checkpoint": MODEL_CHECKPOINT,
        "num_classes": len(label2id),
        "image_size": IMAGE_SIZE,
        "num_epochs": training_args.num_train_epochs,
        "batch_size": training_args.per_device_train_batch_size,
        "learning_rate": training_args.learning_rate,
        "train_samples": len(train_dataset),
        "val_samples": len(val_dataset),
        "test_samples": len(test_dataset)
    })
    
    # Custom data collator for multi-class classification
    def collate_fn(examples):
        import torch
        pixel_values = torch.stack([e["image"] for e in examples])
        labels = torch.tensor([label2id[e["label"]] for e in examples], dtype=torch.long)
        return {"pixel_values": pixel_values, "labels": labels}
    
    # Create trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=feature_extractor,
        data_collator=collate_fn
    )
    
    # Train the model
    print("🏋️ Training model...")
    train_results = trainer.train()
    
    # Log training metrics
    mlflow.log_metrics({
        "train_loss": train_results.training_loss,
        "train_runtime": train_results.metrics["train_runtime"],
        "train_samples_per_second": train_results.metrics["train_samples_per_second"]
    })
    
    # Create inference pipeline
    classifier = pipeline(
        "image-classification",
        model=trainer.state.best_model_checkpoint,
        tokenizer=feature_extractor
    )
    
    # Test the wrapper and create signature
    print("🧪 Testing model wrapper...")
    test_sample = test_data.limit(3).toPandas()
    wrapped_model = CuisineClassifierWrapper(classifier)
    test_predictions = wrapped_model.predict(None, test_sample)
    
    # Create MLflow signature
    signature = infer_signature(test_sample, test_predictions)
    
    # Log the model
    print("📝 Logging model to MLflow...")
    logged_model = mlflow.pyfunc.log_model(
        artifact_path="cuisine_classifier",
        python_model=wrapped_model,
        pip_requirements=mlflow.transformers.get_default_pip_requirements(model),
        signature=signature,
        registered_model_name=f"{CATALOG}.{GOLD_SCHEMA}.cuisine_classifier"
    )
    
    print(f"✅ Model training complete!")
    print(f"   📝 Model URI: {logged_model.model_uri}")
    print(f"   🆔 Run ID: {run.info.run_id}")

## 5. Model Registration and Serving Setup
Register the model for production serving with proper versioning.

In [ ]:
# Register model and set alias for serving
print("📋 Registering model for serving...")

model_name = f"{CATALOG}.{GOLD_SCHEMA}.cuisine_classifier"
client = MlflowClient()

# Get the latest version and set production alias
latest_version = client.get_latest_versions(model_name)[0]
client.set_registered_model_alias(
    name=model_name,
    alias="prod",
    version=latest_version.version
)

print(f"✅ Model registered:")
print(f"   🏷️ Name: {model_name}")
print(f"   📝 Version: {latest_version.version}")
print(f"   🚀 Alias: prod")

print("✅ Model ready for serving via Databricks Model Serving!")

## 6. Batch Inference and Evaluation  
Run batch inference on test data and evaluate model performance.

In [ ]:
# Create UDF for batch inference
print("🔮 Creating inference UDF...")

predict_cuisine_udf = mlflow.pyfunc.spark_udf(
    spark, 
    model_uri=f"models:/{model_name}@prod"
)

# Get input columns for UDF
input_columns = predict_cuisine_udf.metadata.get_input_schema().input_names()

print(f"✅ Inference UDF created:")
print(f"   📊 Input columns: {input_columns}")

In [ ]:
# Run batch inference on test data
print("🧪 Running batch inference on test data...")

test_predictions = (
    test_data
    .withColumn("cuisine_prediction", predict_cuisine_udf(*input_columns))
    .select(
        "image_id", 
        "cuisine_category", 
        "food_type",
        col("cuisine_prediction.label").alias("predicted_cuisine"),
        col("cuisine_prediction.score").alias("confidence_score")
    )
)

# Save predictions to gold layer
test_predictions.write.mode("overwrite").saveAsTable(f"{CATALOG}.{GOLD_SCHEMA}.test_predictions")

print("✅ Batch inference complete")

# Show sample predictions
print("📊 Sample predictions:")
test_predictions.orderBy(col("confidence_score").desc()).show(10)

## 7. Model Evaluation and Metrics
Calculate comprehensive evaluation metrics and create visualizations.

In [ ]:
# Calculate evaluation metrics
print("📈 Calculating evaluation metrics...")

# Convert to pandas for detailed analysis
results_df = test_predictions.toPandas()

# Calculate accuracy
accuracy = (results_df['cuisine_category'] == results_df['predicted_cuisine']).mean()
print(f"🎯 Overall Accuracy: {accuracy:.3f}")

# Per-cuisine accuracy
cuisine_accuracy = results_df.groupby('cuisine_category').apply(
    lambda x: (x['cuisine_category'] == x['predicted_cuisine']).mean()
).sort_values(ascending=False)

print("\n🌍 Per-Cuisine Accuracy:")
print(cuisine_accuracy)

# Log metrics to MLflow
with mlflow.start_run(run_id=run.info.run_id):
    mlflow.log_metric("test_accuracy", accuracy)
    for cuisine, acc in cuisine_accuracy.items():
        mlflow.log_metric(f"accuracy_{cuisine}", acc)

In [ ]:
# Create confusion matrix
print("📊 Creating confusion matrix...")

# Create confusion matrix
confusion_matrix = pd.crosstab(
    results_df['cuisine_category'], 
    results_df['predicted_cuisine'],
    margins=True
)

# Plot confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(
    confusion_matrix.iloc[:-1, :-1],  # Exclude margins
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=confusion_matrix.columns[:-1],
    yticklabels=confusion_matrix.index[:-1]
)
plt.title('Cuisine Classification Confusion Matrix')
plt.xlabel('Predicted Cuisine')
plt.ylabel('Actual Cuisine')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Save confusion matrix as MLflow artifact
confusion_matrix.to_csv("/tmp/confusion_matrix.csv")
with mlflow.start_run(run_id=run.info.run_id):
    mlflow.log_artifact("/tmp/confusion_matrix.csv")

In [ ]:
# Performance analysis by confidence score
print("🎯 Analyzing performance by confidence score...")

# Confidence distribution
plt.figure(figsize=(10, 6))
plt.hist(results_df['confidence_score'], bins=20, alpha=0.7, edgecolor='black')
plt.xlabel('Confidence Score')
plt.ylabel('Frequency')
plt.title('Distribution of Model Confidence Scores')
plt.grid(True, alpha=0.3)
plt.show()

# Accuracy by confidence threshold
thresholds = np.arange(0.1, 1.0, 0.1)
accuracy_by_threshold = []

for threshold in thresholds:
    high_conf_mask = results_df['confidence_score'] >= threshold
    if high_conf_mask.sum() > 0:
        high_conf_accuracy = (
            results_df[high_conf_mask]['cuisine_category'] == 
            results_df[high_conf_mask]['predicted_cuisine']
        ).mean()
        accuracy_by_threshold.append({
            'threshold': threshold,
            'accuracy': high_conf_accuracy,
            'sample_count': high_conf_mask.sum(),
            'coverage': high_conf_mask.mean()
        })

threshold_df = pd.DataFrame(accuracy_by_threshold)
print("\n📊 Accuracy vs Confidence Threshold:")
print(threshold_df)

## 8. Production Inference Example
Demonstrate real-time inference on new images from the gold layer.

In [ ]:
# Example: Inference on new images
print("🔮 Running inference on sample images...")

# Get sample images from different cuisines
sample_images = (
    spark.table(f"{CATALOG}.{GOLD_SCHEMA}.ml_dataset")
    .filter(col("dataset_split") == "test")
    .groupBy("cuisine_category")
    .agg({"*": "first"})  # Get one sample per cuisine
    .select("cuisine_category", "processed_image_data", "food_type")
)

# Run inference
sample_predictions = (
    sample_images
    .withColumn("cuisine_prediction", predict_cuisine_udf("processed_image_data"))
    .select(
        "cuisine_category",
        "food_type", 
        col("cuisine_prediction.label").alias("predicted_cuisine"),
        col("cuisine_prediction.score").alias("confidence_score")
    )
)

print("🧪 Sample inference results:")
sample_predictions.show()

# Save sample predictions
sample_predictions.write.mode("overwrite").saveAsTable(f"{CATALOG}.{GOLD_SCHEMA}.sample_predictions")

## 9. Pipeline Summary and Next Steps

### ✅ Completed Tasks
- **Data Pipeline**: Complete medallion architecture (Bronze → Silver → Gold)
- **Feature Engineering**: 100D feature vectors with advanced image analysis
- **Feature Store**: Advanced features registered for ML discovery and reuse
- **Model Training**: Fine-tuned ResNet-50 for cuisine classification
- **MLflow Integration**: Experiment tracking and model registry
- **Model Serving**: Production-ready model with inference UDF
- **Evaluation**: Comprehensive performance metrics and visualizations

### 🏪 Feature Store Integration
- **Feature Discovery**: Browse all image features in Databricks UI catalog
- **Feature Lineage**: Track how features are created from DLT pipeline
- **Feature Serving**: Fast lookups for real-time inference
- **Feature Reuse**: Share 100D vectors and color/texture features across teams
- **Feature Monitoring**: Built-in drift detection and quality monitoring

### 🚀 Next Steps
1. **Model Serving Endpoint**: Deploy model to Databricks Model Serving
2. **Real-time Inference**: Create REST API for real-time predictions with Feature Store lookups
3. **Model Monitoring**: Set up drift detection and performance monitoring
4. **A/B Testing**: Compare model versions for continuous improvement
5. **Advanced Features**: Leverage Feature Store for feature experimentation and versioning

### 🎯 Production Architecture
```
HuggingFace → DLT Pipeline → Feature Store → MLflow → Model Serving
    ↓              ↓             ↓           ↓           ↓
Raw Images → Gold Tables → Features → Models → Inference
```

In [ ]:
# Final pipeline summary
print("🎉 CUISINE CLASSIFICATION PIPELINE COMPLETE!")
print("=" * 60)
print("📊 Pipeline Summary:")
print(f"   🥉 Bronze: Raw image data from HuggingFace")
print(f"   🥈 Silver: 8 cuisine tables with 224x224 processed images") 
print(f"   🥇 Gold: ML-ready datasets with 100D feature vectors")
print(f"   🧠 Model: Fine-tuned ResNet-50 for {len(label2id)} cuisines")
print(f"   🎯 Accuracy: {accuracy:.1%}")
print(f"   📊 Test Samples: {len(results_df)}")
print(f"   🚀 Model: {model_name}@prod")
print("=" * 60)
print("✅ Ready for production deployment!")